In [ ]:
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,Callback


from tensorflow.keras.optimizers import Adam
import keras
import tensorflow as tf
from keras import backend as k
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from sklearn.model_selection import train_test_split
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt, os
warnings.filterwarnings('ignore')

#object serialization
import _pickle as cPickle  #python 3 change
import os  

%matplotlib inline

In [ ]:
!unzip /kaggle/input/freesound-audio-tagging-2019/train_curated.zip

In [ ]:
import pandas as pd
import tqdm as tqdm
import pandas as pd
import numpy as np
import os
import librosa

label = pd.read_csv(r'C:\Users\Ramstein\Documents\SmartsuitStudioProjects\Sample project\Export\scene-1\SuddenCrouch_HUMANIK_E46.csv')
df = label.drop(['Timestamp'], axis=1)
dfn = df.to_numpy().astype(np.float32)
print(dfn.shape)
np.save('label.npy', dfn)

In [ ]:
import pandas as pd
import tqdm as tqdm
import pandas as pd
import numpy as np
import os
import librosa

def data_to_spec(data):
    d = librosa.stft(data)   # wav to short-term-fourier-transform
    return librosa.power_to_db(np.abs(d)**2, ref=np.median)

def file_to_spec(filename, i):
    y, sr = librosa.load(filename,sr=16000, offset=(0.03333333333)*i, duration=0.03333333333)
    print(sr)
    return data_to_spec(y)

Path = r'C:\Conv\wavs'
# files = os.listdir(path)
files=['Conv12.wav']
arr = []

for file in tqdm.tqdm(files):
    file = os.path.join(Path, file)
    duration = librosa.core.get_duration(filename=file) 
    print('duration:', duration )
    Slice=int((duration*1000)//33.3333333333)

    for i in range(Slice):
        try:
            d = file_to_spec(file, i)
            d = d.transpose()
        except Exception as e:print(e);continue
        arr.append(d)
    break

data_n = np.array(arr) /255
print(data_n.shape)
# np.save('train.npy', data_n[:652])

In [ ]:
label = pd.read_csv('../input/motion-sudden-crouch/SuddenCrouch_HUMANIK_E46.csv')
df = label.drop(['Timestamp'], axis=1)
df.head()
# lanel reshaping and converting into the classes
dfn = df.to_numpy().astype(np.float16)
dfn = dfn[:-1]
print(dfn.shape, dfn[0].shape)  #dfn.shape=(651, 441) where 651 is total number of samples
                                # and 441 is number of classes

In [ ]:
sr = 16000
# for i, file in enumerate(files):
# if i==20: break    
fname = os.path.join('/kaggle/working', files[14])   # Hi-hat
wav, _ = librosa.core.load(fname)

wav = wav[:330]
#display waveform

#     plt.figure(figsize=(14, 5))
#     librosa.display.waveplot(wav, sr=sr)


#display Spectrogram
X = librosa.stft(wav)
Xdb = librosa.amplitude_to_db(abs(X))
print(Xdb)
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
#If to pring log of frequencies  
#librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()



In [ ]:
# train = np.load('/kaggle/input/audio-mfcc-test/train.npy')
# dfn = np.load('/kaggle/input/audio-mfcc-test/label.npy')


# #split up test into test and validation 
# X_train, X_val, y_train, y_val = train_test_split(train, dfn, test_size=0.30, random_state=42)

# X_train  = X_train.reshape(X_train.shape[0], 33, 1025, 1)
# X_val  = X_val.reshape(X_val.shape[0], 33, 1025, 1)


# X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.40, random_state=42)

# print ("test ",X_train.shape, len(y_train))
# print ("valid ", X_val.shape, len(y_val))

In [ ]:
import collections
import math
import string, random, time

import tensorflow as tf
from six.moves import xrange
from tensorflow.keras import backend as backend
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Dense, Reshape, Input
from tensorflow.keras.layers import multiply, add
from tensorflow.keras.models import Model

BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'strides', 'se_ratio'
])
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)
DEFAULT_BLOCKS_ARGS = [
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,
              expand_ratio=1, id_skip=True, strides=1, se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,
              expand_ratio=6, id_skip=True, strides=2, se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,
              expand_ratio=6, id_skip=True, strides=2, se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,
              expand_ratio=6, id_skip=True, strides=2, se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,
              expand_ratio=6, id_skip=True, strides=1, se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,
              expand_ratio=6, id_skip=True, strides=2, se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,
              expand_ratio=6, id_skip=True, strides=1, se_ratio=0.25)
]

CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        # EfficientNet1D actually uses an untruncated normal distribution for
        # initializing conv layers, but keras.initializers.VarianceScaling use
        # a truncated distribution.
        # We decided against a custom initializer for better serializability.
        'distribution': 'normal'
    }
}
DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}


def round_filters(filters, width_coefficient, depth_divisor):
    """Round number of filters based on width multiplier."""
    filters *= width_coefficient
    new_filters = int(filters + depth_divisor / 2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    # Make sure that round down does not go down by more than 10%.
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)


def get_swish(**kwargs):
    def swish(x):
        """Swish activation function: x * sigmoid(x).
        Reference: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
        """
        if backend.backend() == 'tensorflow':
            try:
                # The native TF implementation has a more
                # memory-efficient gradient implementation
                return tf.nn.swish(x)
            except AttributeError:
                pass
        return x * backend.sigmoid(x)

    return swish


def round_repeats(repeats, depth_coefficient):
    """Round number of repeats based on depth multiplier."""
    return int(math.ceil(depth_coefficient * repeats))


def get_dropout(**kwargs):
    """Wrapper over custom dropout. Fix problem of ``None`` shape for tf.keras.
    It is not possible to define FixedDropout class as global object,
    because we do not have modules for inheritance at first time.
    Issue:
        https://github.com/tensorflow/tensorflow/issues/30946
    """

    class FixedDropout(tf.keras.layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape
            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout


def mb_conv_block(inputs, block_args, activation, drop_rate=None, prefix='', ):
    has_se = (block_args.se_ratio is not None) and (0 < block_args.se_ratio <= 1)
    # workaround over non working dropout with None in noise_shape in tf.keras
    Dropout = get_dropout()
    d_num = 1
    # expansion phase
    filters = block_args.input_filters * block_args.expand_ratio
    if block_args.expand_ratio != 1:
        x = Conv1D(filters,
                   kernel_size=1,
                   padding='causal', use_bias=False,
                   kernel_initializer=CONV_KERNEL_INITIALIZER,
                   name=prefix + 'expand_conv')(inputs)
        x = tf.keras.layers.Dropout(rate=0.05, name='exp_dropout-{}'.format(random.randint(1, 10000)))(x);random.seed(time.clock())
        x = Activation(activation, name=prefix + 'expand_activation-{}'.format(random.randint(1, 10000)))(x);random.seed(time.clock())
    else:
        x = inputs

    # Depthwise Convolution
    x = Conv1D(filters=filters,
               kernel_size=block_args.kernel_size,
               strides=block_args.strides,
               padding='causal',
               use_bias=False,
               kernel_initializer=CONV_KERNEL_INITIALIZER,
               name=prefix + 'dw_conv')(x)
    x = tf.keras.layers.Dropout(rate=0.05, name=prefix + 'exp_dropout-{}'.format(random.randint(1, 10000)))(x);random.seed(time.clock())
    x = Activation(activation, name=prefix + 'expand_activation-{}'.format(random.randint(1, 10000)))(x);random.seed(time.clock())

    # squeeze the excitation phase
    if has_se:
        num_reduced_filters = max(1, int(block_args.input_filters * block_args.se_ratio))
        se_tensor = GlobalAveragePooling1D(name=prefix + 'se_squeeze')(x)

        ###########################look here
        target_shape = (1, filters)
        se_tensor = Reshape(target_shape, name=prefix + 'se_reshape')(se_tensor)
        se_tensor = Conv1D(filters=num_reduced_filters,
                           kernel_size=1,
                           activation=activation,
                           padding='causal',
                           use_bias=True,
                           kernel_initializer=CONV_KERNEL_INITIALIZER,
                           name=prefix + 'se_reduce')(se_tensor)
        se_tensor = Conv1D(filters=filters,
                           kernel_size=1,
                           activation='sigmoid',
                           padding='causal',
                           use_bias=True,
                           kernel_initializer=CONV_KERNEL_INITIALIZER,
                           name=prefix + 'se_expand')(se_tensor)
        x = multiply([x, se_tensor], name=prefix + 'se_excite')

    # Output phase
    x = Conv1D(filters=block_args.output_filters,
               kernel_size=1,
               padding='causal',
               use_bias=False,
               kernel_initializer=CONV_KERNEL_INITIALIZER,
               name=prefix + 'project_conv')(x)
    x = tf.keras.layers.Dropout(rate=0.05, name=prefix + 'out_phase_dropout')(x)
    if block_args.id_skip and block_args.input_filters == block_args.output_filters:
        if drop_rate and (drop_rate > 0):
            x = Dropout(drop_rate,
                        noise_shape=(None, 1, 1),
                        name=prefix + 'drop')(x)
        x = add([x, inputs], name=prefix + 'add')

    return x


def EfficientNet1D(width_coefficient,
                   depth_coefficient,
                   default_resolution,
                   dropout_rate=0.2,
                   drop_connect_rate=0.2,
                   depth_divisor=8,
                   blocks_args=DEFAULT_BLOCKS_ARGS,
                   model_name='efficientnet',
                   include_top=True,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    """Instantiates the EfficientNet1D architecture using given scaling coefficients.
        Optionally loads weights pre-trained on ImageNet.
        Note that the data format convention used by the model is
        the one specified in your Keras config at `~/.keras/keras.json`.
        # Arguments
            width_coefficient: float, scaling coefficient for network width.
            depth_coefficient: float, scaling coefficient for network depth.
            default_resolution: int, default input image size.
            dropout_rate: float, dropout rate before final classifier layer.
            drop_connect_rate: float, dropout rate at skip connections.
            depth_divisor: int.
            blocks_args: A list of BlockArgs to construct block modules.
            model_name: string, model name.
            include_top: whether to include the fully-connected
                layer at the top of the network.

            input_shape: optional shape tuple, only to be specified
                if `include_top` is False.
                It should have exactly 3 inputs channels.
            pooling: optional pooling mode for feature extraction
                when `include_top` is `False`.
                - `None` means that the output of the model will be
                    the 4D tensor output of the
                    last convolutional layer.
                - `avg` means that global average pooling
                    will be applied to the output of the
                    last convolutional layer, and thus
                    the output of the model will be a 2D tensor.
                - `max` means that global max pooling will
                    be applied.
            classes: optional number of classes to classify images
                into, only to be specified if `include_top` is True, and
                if no `weights` argument is specified.
        # Returns
            A Keras model instance.
        # Raises
            ValueError: in case of invalid argument for `weights`,
                or invalid input shape.
        """
    activation = get_swish(**kwargs)
    input = Input(shape=input_shape)
    # Build stem
    x = input
    x = Conv1D(filters=round_filters(32, width_coefficient, depth_divisor),
               kernel_size=3,
               strides=2,
               padding='causal',
               use_bias=False,
               kernel_initializer=CONV_KERNEL_INITIALIZER,
               name='stem_conv')(input)
    x = tf.keras.layers.Dropout(rate=0.05, name='stem_dropout')(x)
    x = Activation(activation, name='stem_activation', )(x)

    # Build blocks
    num_blocks_total = sum(block_args.num_repeat for block_args in blocks_args)
    block_num = 0
    for idx, block_args in enumerate(blocks_args):
        assert block_args.num_repeat > 0
        # Update block input and output filters based on depth multiplier.
        block_args = block_args._replace(
            input_filters=round_filters(block_args.input_filters,
                                        width_coefficient, depth_divisor),
            output_filters=round_filters(block_args.output_filters,
                                         width_coefficient, depth_divisor),
            num_repeat=round_repeats(block_args.num_repeat, depth_coefficient))

        # The first block needs to take care of stride and filter size increase.
        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = mb_conv_block(x, block_args,
                          activation=activation,
                          drop_rate=drop_rate,
                          prefix='block{}a_'.format(idx + 1))
        block_num += 1
        if block_args.num_repeat > 1:
            # pylint: disable=protected-access
            block_args = block_args._replace(
                input_filters=block_args.output_filters, strides=1)
            # pylint: enable=protected-access
            for bidx in xrange(block_args.num_repeat - 1):
                drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
                block_prefix = 'block{}{}_'.format(
                    idx + 1,
                    string.ascii_lowercase[bidx + 1]
                )
                x = mb_conv_block(x, block_args,
                                  activation=activation,
                                  drop_rate=drop_rate,
                                  prefix=block_prefix)
                block_num += 1

    # Build top
    x = Conv1D(round_filters(1280, width_coefficient, depth_divisor), 1,
               padding='same',
               use_bias=False,
               kernel_initializer=CONV_KERNEL_INITIALIZER,
               name='top_conv')(x)
    x = tf.keras.layers.Dropout(rate=0.05, name='top_dropout-{}'.format(random.randint(1, 10000)))(x);random.seed(time.clock())
    x = Activation(activation, name='top_activation')(x)

    if include_top:
        x = GlobalAveragePooling1D(name='avg_pool')(x)
        if dropout_rate and dropout_rate > 0:
            x = tf.keras.layers.Dropout(dropout_rate, name='top_dropout-{}'.format(random.randint(1, 10000)))(x);random.seed(time.clock())
        x = Dense(classes,
                  activation='softmax',
                  kernel_initializer=DENSE_KERNEL_INITIALIZER,
                  name='probs')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling1D(name='avg_pool')(x)
        elif pooling == 'max':
            x = GlobalMaxPooling1D(name='max_pool')(x)

    # Create model.
    model = Model(input, x, name=model_name)
    return model


def EfficientNetB0(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.0, 1.0, 224, 0.2,
                          model_name='efficientnet-b0',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB1(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.0, 1.1, 240, 0.2,
                          model_name='efficientnet-b1',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB2(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.1, 1.2, 260, 0.3,
                          model_name='efficientnet-b2',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB3(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.2, 1.4, 300, 0.3,
                          model_name='efficientnet-b3',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB4(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.4, 1.8, 380, 0.4,
                          model_name='efficientnet-b4',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB5(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.6, 2.2, 456, 0.4,
                          model_name='efficientnet-b5',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB6(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(1.8, 2.6, 528, 0.5,
                          model_name='efficientnet-b6',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


def EfficientNetB7(include_top=True,input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet1D(2.0, 3.1, 600, 0.5,
                          model_name='efficientnet-b7',
                          include_top=include_top, input_shape=input_shape,
                          pooling=pooling, classes=classes,
                          **kwargs)


In [ ]:

model = EfficientNetB3(include_top=True,
                   input_shape=(2, 1025),
                   pooling='max',
                   classes=441)
model.summary()
%%time

# NOTE:
# Increase number if epochs from  1 to 60 or 100 for higher prediction accuracy
# default is set to 1 for faster commit 
MAX_EPOCHS=100
MAX_BATCH_SIZE=48            
# learning rate reduction rate 
MAX_PATIENT=10

# saved model checkpoint file
best_model_file="./best_model_trained.hdf5"

# callbacks
# removed EarlyStopping(patience=MAX_PATIENT)
callback=[ReduceLROnPlateau(patience=MAX_PATIENT, verbose=1),
          EarlyStopping(monitor='loss', mode='max', verbose=1, patience=50),
          ModelCheckpoint(filepath=best_model_file, monitor='loss', verbose=1, save_best_only=True)]

#compile
model.compile(optimizer="Adam",loss="mean_squared_error",metrics=['accuracy', 'mean_squared_error'])


In [ ]:
# model = load_model('/kaggle/input')

In [ ]:
audio_stft_files = []
transform_files = []
path = '/kaggle/input'

if len(audio_stft_files)==len(transform_files): data = list(zip(audio_stft_files, transfrom_files))
for a_file, t_file in data:
    audio = np.load(os.path.join(path, a_file)
    transform = np.load(os.path.join(path, t_file)
    print(audio.shape, transform.shape)
                        
                        
                        
    X_train, X_val, y_train, y_val = train_test_split(audio, transform, test_size=0.30, random_state=42)
    print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
    
                        
    #train
    print('training started.... please wait!')
    history = model.fit(x=X_train, y=y_train,
                        epochs=MAX_EPOCHS,
                        batch_size=MAX_BATCH_SIZE, 
                        verbose=1,
                        validation_data= (X_val, y_val), 
                        callbacks=callback)
    print('training finished')

    # # quick evaludate model
    # print('Evaluate model with test data')
    # score = model.evaluate(x=X_test,y=y_test)


In [ ]:
%%time
# %%memit

import matplotlib.pyplot as plt
#Plot loss and accuracy for the training and validation set.
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    plt.figure(figsize=(22,10))
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    ## Accuracy
    plt.figure(221, figsize=(20,10))
    ## Accuracy
    # plt.figure(2,figsize=(14,5))
    plt.subplot(221, title='Accuracy')
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    ## Loss
    plt.subplot(222, title='Loss')
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# plot history
plot_history(history)